
# SixtyFour AI — Enrich Lead

This notebook is your guide to using the **Sixtyfour API `enrich-lead`**. The enrich_lead endpoint allows you to perform deep research on an individual—be it a prospect, candidate, or contact.

Enrich-Lead recrusively reads through every link and with this knowledge it can answer questions like: What is this individual an expert in? How many publications do they have? Are they a fashion reseller? Do they have experience with M&A transactions? Etc.

Anything that you can think of is fair game for our API. Please give it a go, and let us know how we can help!

> **Tip:** You can change the person in the parameters at the top. It can be **anyone with a digital presence**. For best results, include **at least**: `name` **and** `company`, or a direct identifier like `email` **or** `linkedin`.



## Prerequisites & Secrets

### Get your SixtyFour API key

Follow these steps to create and copy your API key

**1) Sign in**
Go to <https://app.sixtyfour.ai/login> and log in or sign up.

**2)** Enter an organization name once you've signed up and click **Create Organization**.

**3)** You’ll land on the dashboard/usage page with $25 in credits. In the left sidebar, open **Keys** → click **Create new key** → name it → **Create key** → copy the key value.



### If you’re in **Google Colab**
1. Click the **🔑 (Secrets)** icon in the left sidebar.  
2. Add a secret named `SIXTYFOUR_API_KEY` and paste your key.  
3. Make sure the notebook can access the secret.


In [ ]:

# --- Parameters (you can change these) ---
# Only name+company/university OR linkedin is needed
# You can also give the agent more context following the structure below
LEAD_NAME = "Saarth Shah"
LEAD_COMPANY = "Sixtyfour AI"
LEAD_EMAIL = None
LEAD_LINKEDIN = None


In [ ]:

import os, json, time, csv, io
from datetime import datetime
from typing import Dict, Any, List, Optional

import requests
import pandas as pd

API_KEY = os.getenv("SIXTYFOUR_API_KEY", "")  # fallback if already set locally
try:
    from google.colab import userdata
    secret_val = userdata.get("SIXTYFOUR_API_KEY")  # raises if not granted or not created
    if secret_val:  # avoid overwriting with empty
        os.environ["SIXTYFOUR_API_KEY"] = secret_val
        API_KEY = secret_val
except Exception:
    # Not in Colab or secret not available; rely on existing env var
    pass

BASE_URL = "https://api.sixtyfour.ai"
SYNC_ENDPOINT = f"{BASE_URL}/enrich-lead"
ASYNC_ENDPOINT = f"{BASE_URL}/enrich-lead-async"
STATUS_ENDPOINT = f"{BASE_URL}/job-status"
TIMEOUT_S = 600  # We reuse these endpoints and TIMEOUT_S throughout the notebook.

print("✅ Setup complete (key detected, endpoints ready).")


✅ Setup complete (key detected, endpoints ready).


## Enrich_Lead Request

For most queries, enrich_lead is the perfect tool. You send a request and get a complete profile back with a single call.

The request requires two main components in its body:

* **`lead_info`**: The initial data you have on the person. The more context you provide, the more focused and correct the research will be.
* **`struct`**: Here is where you can request the niche pieces of information you'd like. The first part "name" is the field you want filled. The second part following the ":" is the description given to the agent to help fulfill your exact request.

Feel free to change structure in "struct" to something more relevant to you!

**WARNING**  Please expect a four to five minute wait time for your response! We are working on latentcy, but the agent is actively reading everything live, which take a while

In [ ]:

lead_info = {
    "name": LEAD_NAME,
    "company": LEAD_COMPANY,
    # "email": LEAD_EMAIL,        # ← uncomment if you provided it above
    # "linkedin": LEAD_LINKEDIN,  # ← uncomment if you provided it above
}

# Ask the API for a few fields; uncomment or add more if you want to find additional data.
struct = {
    "name": "The individual's full name",
    "email": {"description": "Work email", "type": "string"},
    "linkedin": "LinkedIn URL",
    "location": "Location (city, region, country)",
    "title": "Current job title",
    "company_size": "Company size (employees)",
    "industry": "Industry of the company",
    "twitter_url": "Twitter profile URL",
    "github_url": "GitHub profile URL",
    # You can also add your own fields here to find more information about the lead.


    # You can ask for typed fields (predictable format). Types (int, boolean, list[string], etc) make downstream code easier.
    "email": {"description": "Work email", "type": "string"},
    "previous_companies": {"description": "Previous employers", "type": "list[string]"},
    "years_experience": {"description": "Total years", "type": "int"},
    "is_executive": {"description": "Is executive role", "type": "boolean"}
}

# This is the core API call to enrich a lead.
resp = requests.post(
    SYNC_ENDPOINT,
    headers={"x-api-key": API_KEY, "Content-Type": "application/json"},
    json={"lead_info": lead_info, "struct": struct},
    timeout=TIMEOUT_S
)

if resp.ok:
    data = resp.json()
    print("✅ Success! Top-level keys:", list(data.keys()))
    print("\nStructured data:")
    print(json.dumps(data.get("structured_data", {}), indent=2))
else:
    print("❌ Request failed:", resp.status_code, resp.text[:500])

✅ Success! Top-level keys: ['notes', 'structured_data', 'findings', 'references', 'confidence_score']

Structured data:
{
  "name": "Saarth Shah",
  "email": "saarth@sixtyfour.ai",
  "linkedin": "https://www.linkedin.com/in/saarthshah/",
  "location": "San Francisco Bay Area, California, United States",
  "title": "Co-founder and CEO",
  "company_size": "10-50 (estimated)",
  "industry": "Artificial Intelligence / Sales technology (SaaS)",
  "twitter_url": "https://x.com/saarth_",
  "github_url": "https://github.com/SaarthShah",
  "previous_companies": "['Whatnot', 'Deepgram', 'Internalize']",
  "years_experience": 8,
  "is_executive": true,
  "company": "Sixtyfour AI"
}



## Example structs you can run (copy/paste)

Pick one of these, assign to `struct`, and make the same `requests.post` call as above.


In [ ]:

# Sales prospecting
example_sales_struct = {
    "full_name": "Full name",
    "title": "Current job title",
    "email": "Professional email",
    "phone": "Phone number",
    "linkedin": "LinkedIn URL",
    "decision_authority": "Budget/purchasing authority",
    "team_size": {"description": "Size of team managed", "type": "int"},
    "recent_funding": "Recent funding news",
    "pain_points": "Known business challenges",
    "best_contact_time": "Best time to contact"
}

# Recruiting
example_recruiting_struct = {
    "full_name": "Full name",
    "current_title": "Current job title",
    "years_experience": {"description": "Total years", "type": "int"},
    "technical_skills": {"description": "Technical skills", "type": "list[string]"},
    "previous_companies": {"description": "Previous employers", "type": "list[string]"},
    "education": "Highest degree",
    "open_to_opportunities": {"description": "Open to new roles", "type": "boolean"},
    "salary_expectations": "Expected salary range"
}

# Marketing
example_marketing_struct = {
    "full_name": "Full name",
    "title": "Job title",
    "company_size": "Company size (employees)",
    "budget_authority": {"description": "Has budget authority", "type": "boolean"},
    "content_topics": {"description": "Topics they post about", "type": "list[string]"},
    "social_influence": "Social media influence level",
    "purchase_timeline": "Typical purchase timeline",
    "preferred_channels": "Preferred communication channels"
}


# Business Development / Partnerships
example_bd_struct = {
    "full_name": "Full name",
    "title": "Role/title",
    "decision_authority": "Partnership decision authority",
    "strategic_initiatives": {"description": "Key initiatives they own", "type": "list[string]"},
    "past_partnerships": {"description": "Notable past partnerships", "type": "list[string]"},
    "regions": {"description": "Regions covered", "type": "list[string]"},
    "best_contact_time": "Best time to contact"
}

# Investor / VC Outreach
example_investor_struct = {
    "full_name": "Full name",
    "focus_sectors": {"description": "Sectors of interest", "type": "list[string]"},
    "stage_focus": "Stages (pre-seed/seed/A/B/...)",
    "check_size": "Typical check size",
    "notable_investments": {"description": "Notable investments", "type": "list[string]"},
    "recent_funds": "Recent fund announcements",
    "location": "Location"
}



## Troubleshooting & Next steps

**Remember:**  
- You can request up to 100 fields in one `struct`  
- Adding more fields doesn't increase API usage  
- Rate limit: **500 requests per minute** per API key  
- Don't be vague — “Saarth” vs “Saarth Shah, CEO at Sixtyfour AI”  
- Provide context — More details = better results  


**If a request fails:** make sure you provide either `email`, `linkedin`, or `(name AND company)`.  
If fields are missing, request fewer first or add more context to `lead_info`. For large workloads, prefer async or the concurrent batch.

**Next steps:** try a different `struct`, point the CSV output to your CRM, or move this into an automated script.

---

*This notebook stays simple on purpose: small cells, clear comments, and the same pattern repeated so anyone can follow.*
